!pip install kaleido
import kaleido #required
kaleido.__version__ #0.2.1

import plotly
plotly.__version__ #5.5.0

!pip install gitly
from gitly.colab.plot import GitlyPlotter
gitly = GitlyPlotter('github')

In [78]:
import plotly.offline as py
py.init_notebook_mode(connected=True)

In [79]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
import json
import numpy
from IPython.display import display
from subprocess import check_output
import sys
import traceback
from tqdm.notebook import tqdm, trange
import plotly.graph_objects as go
import plotly.express as px
from paramiko import SSHClient
from scp import SCPClient
from subprocess import call
#import plotly.io as pio
#pio.renderers.default = "svg"

In [73]:
techs = {
    "tech1": "1q_1o",
    "tech2": "k_per_terra",
    "tech3": "4pip",
    "tech4": "7vp",
    "tech5": "1o_1pw",
    "tech6": "1k_1c",
    "tech7": "3vp_per_gaia_place",
    "tech8": "4c",
    "tech9": "4pw",
    # idk wtf this is --> "tech-ship0"
}

adv_techs = {
    "advtech1": "3vp_per_fed_pass",
    "advtech2": "2vp_per_tech_bump",
    "advtech3": "1qic_5c_action",
    "advtech4": "2vp_per_mine",
    "advtech5": "3vp_per_rl_pass",
    "advtech6": "1o_per_sector",
    "advtech7": "1vp_per_terra_pass",
    "advtech8": "2vp_per_gaia",
    "advtech9": "4vp_per_ts",
    "advtech10": "2vp_per_sector",
    "advtech11": "3o_action",
    "advtech12": "5vp_per_fed",
    "advtech13": "3k_action",
    "advtech14": "3vp_per_mine_place",
    "advtech15": "3vp_per_ts_place",
}

feds = {
    "fed1": "12vp",
    "fed2": "qic",
    "fed3": "2pw",
    "fed4": "2o",
    "fed5": "6c",
    "fed6": "2k",
    "gleens": "gleens"
}

round_scorings = {
    "score1": "2vp_per_terra",
    "score2": "2vp_per_research_bump",
    "score3": "2vp_per_mine_place",
    "score4": "5vp_per_fed_place",
    "score5": "4vp_per_ts_place",
    "score6": "4vp_per_gaia_place",
    "score7": "5vp_per_3pip_place",
    "score8": "3vp_per_ts_place",
    "score9": "3vp_per_gaia_place",
    "score10": "5vp_per_3pip_place",
}

boosters = {

    "booster1": "1k_1o",
    "booster2": "2pwt_1o",
    "booster3": "1qic_2c",
    "booster4": "2c_terra",
    "booster5": "2pw_nav",
    "booster6": "1o_1vp_per_mine",
    "booster7": "1o_2vp_per_ts",
    "booster8": "1k_3vp_per_rl",
    "booster9": "4pw_4vp_per_3pip",
    "booster10": "4c_1vp_per_gaia",
}

bad_buildings = {"colony", 
                 "colonyShip", 
                 "tradeShip", 
                 "constructionShip", 
                 "researchShip",
                 "scout",
                 "frigate",
                 "battleShip",
                 "customsPost",
                 "tradePost"
                }

factions = {
    "terrans",
    "lantids",
    "hadsch-hallas",
    "ivits",
    "baltaks",
    "geodens",
    "xenos",
    "gleens",
    "ambas",
    "taklons",
    "bescods",
    "firaks",
    "itars",
    "nevlas"
}

In [74]:
dat_2 = pd.read_pickle("./data/two_players_data")
dat_3 = pd.read_pickle("./data/three_players_data")
dat_4 = pd.read_pickle("./data/four_players_data")

In [80]:
def plot_adv_freq_remote(num_players, dat_x):
    dat_clean = dat_x.copy()
    for pos in range(num_players):
        dat_clean = dat_clean[dat_clean['pos_' + str(pos + 1) + '_dropped'] == False]

    for faction in factions:
        factions_pos = []
        for pos in range(num_players):
            factions_pos.append(dat_clean[dat_clean["pos_" + str(pos + 1) + "_faction"] == faction])
        factions_in = pd.concat(factions_pos, ignore_index=True)

        fig = go.Figure()

        x = []
        for val in adv_techs.values():
            x.append(val)

        # get overall percents for printing
        overall = [0] * len(adv_techs.values())
        for i in range(num_players):
            for j, tech in enumerate(adv_techs.values()):
                df = factions_pos[i]

                # get all games where tech is present
                slots_df = []
                tech_slots = [
                    "tech_adv-terra",
                    "tech_adv-nav",
                    "tech_adv-int",
                    "tech_adv-gaia",
                    "tech_adv-eco",
                    "tech_adv-sci"
                ]
                for slot in tech_slots:
                    slots_df.append(factions_in[factions_in[slot] == tech])
                df_avail = pd.concat(slots_df, ignore_index=True)

                freq_taken = len(df[df['pos_' + str(i + 1) + '_adv_tech_taken_' + tech] == True]) / len(df_avail)
                overall[j] += freq_taken

        for i in range(num_players):
            y = []
            info = []
            for j, tech in enumerate(adv_techs.values()):
                df = factions_pos[i]

                # get all games where tech is present
                slots_df = []
                tech_slots = [
                    "tech_adv-terra",
                    "tech_adv-nav",
                    "tech_adv-int",
                    "tech_adv-gaia",
                    "tech_adv-eco",
                    "tech_adv-sci"
                ]
                for slot in tech_slots:
                    slots_df.append(factions_in[factions_in[slot] == tech])
                df_avail = pd.concat(slots_df, ignore_index=True)

                freq_taken = len(df[df['pos_' + str(i + 1) + '_adv_tech_taken_' + tech] == True]) / len(df_avail)
                y.append(round(freq_taken * 100, 3))
                info.append(round(((freq_taken / overall[j]) * 100), 3))
            fig.add_trace(go.Bar(x=x, y=y, name='rank_' + str(i + 1),
                                 hovertemplate=
                                 '<br><b>Rank</b>: ' + str(i + 1) +
                                 '<br><b>Tech</b>: %{x}' +
                                 '<br><b>% Taken if Available</b>: %{y}' +
                                 '<br><b>% Rank if Taken</b>: %{text}' +
                                 '<br><b>% Rank overall</b>: ' + str(
                                     round((len(factions_pos[i]) / len(factions_in)) * 100, 2)) +
                                 '<extra></extra>',
                                 text=info,
                                 texttemplate=''))

        title_text = "Frequency Advanced Tech Tile is Taken and the Resulting Rank Distribution: " + faction

        fig.update_layout(uniformtext_minsize=100000, uniformtext_mode='hide')
        fig.update_layout(barmode='stack')
        fig.update_xaxes(categoryorder='total ascending')
        fig.update_layout(title_text=title_text, xaxis_title="Advanced Tech",
                          yaxis_title="Percent of Games Taken")

        #fig.show()
        #gitly.show(fig)
        py.iplot(fig, filename='plot_adv_freq_remote_' + str(faction) + '_' + str(num_players))

In [76]:
def plot_adv_track_remote(num_players, dat_x):
        # do cleaning
    dat_clean = dat_x.copy()
    for pos in range(num_players):
        dat_clean = dat_clean[dat_clean['pos_' + str(pos + 1) + '_dropped'] == False]
        dat_clean['pos_' + str(pos + 1) + '_score'] += abs(dat_clean['pos_' + str(pos + 1) + '_score_bid'])

    for faction in factions:

        tech_slots = [
            "tech_adv-terra",
            "tech_adv-nav",
            "tech_adv-int",
            "tech_adv-gaia",
            "tech_adv-eco",
            "tech_adv-sci"
        ]

        factions_pos = []
        for pos in range(num_players):
            faction_dat = dat_clean[dat_clean["pos_" + str(pos + 1) + "_faction"] == faction]
            factions_pos.append(faction_dat[faction_dat['pos_' + str(pos + 1) + '_score'] >= 130])
        factions_in = pd.concat(factions_pos, ignore_index=True)

        techs = list(adv_techs.values())

        slot_scores = []
        for i in range(len(tech_slots) + 2):
            slot_scores.append([])

        game_nums = []
        for i in range(len(tech_slots) + 2):
            game_nums.append([])

        fig = go.Figure()

        for tech in adv_techs.values():
            tot_taken_score = 0
            tot_taken_slot_count = 0
            tot_taken_amount = 0

            tot_not_taken_score = 0
            not_taken_slot_count = 0
            tot_not_taken_amount = 0
            for i, slot in enumerate(tech_slots):
                taken_score = 0
                taken_amount = 0
                pos_taken = 0
                not_taken_score = 0
                not_taken_amount = 0
                pos_not_taken = 0
                for pos in range(num_players):
                    df = factions_pos[pos]
                    df = df[df[slot] == tech]
                    df_not_taken = df[df['pos_' + str(pos + 1) + "_adv_tech_taken_" + tech] == False]
                    if len(df_not_taken) > 0:
                        pos_not_taken += 1
                        not_taken_score += df_not_taken['pos_' + str(pos + 1) + "_score"].mean()
                        not_taken_amount += len(df_not_taken)
                        tot_not_taken_amount += len(df_not_taken)
                    df_taken = df[df['pos_' + str(pos + 1) + "_adv_tech_taken_" + tech] == True]
                    if len(df_taken) > 0:
                        pos_taken += 1
                        taken_score += df_taken['pos_' + str(pos + 1) + "_score"].mean()
                        taken_amount += len(df_taken)
                        tot_taken_amount += len(df_taken)
                if pos_taken > 0:
                    slot_scores[i].append(taken_score / pos_taken)
                    tot_taken_score += taken_score / pos_taken
                    tot_taken_slot_count += 1
                else:
                    slot_scores[i].append(None)
                if pos_not_taken > 0:
                    tot_not_taken_score += not_taken_score / pos_not_taken
                    not_taken_slot_count += 1

                game_nums[i].append(taken_amount)
            slot_scores[len(tech_slots)].append(tot_not_taken_score / not_taken_slot_count)
            game_nums[len(tech_slots)].append(tot_not_taken_amount)
            slot_scores[len(tech_slots) + 1].append(tot_taken_score / tot_taken_slot_count)
            game_nums[len(tech_slots) + 1].append(tot_taken_amount)

        tech_slots.append("tech_adv-not_taken")
        tech_slots.append("tech_adv-taken")

        colors = [
            'rgba(165, 42, 42, .9)',
            'rgba(112, 128, 144, .9)',
            'rgba(38, 166, 91, .9)',
            'rgba(191, 85, 236, .9)',
            'rgba(255, 215, 0, .9)',
            'rgba(3, 138, 255, .9)',
            'rgba(242, 38, 19, .9)',
            'rgba(0, 230, 64, 1)'

        ]

        for i, slot in enumerate(tech_slots):
            fig.add_trace(go.Scatter(
                x=slot_scores[i],
                y=techs,
                mode="markers",
                marker_color=colors[i],
                marker_symbol=[
                    'triangle-down' if i == 6 else 'triangle-up' if i == 7 else 'circle-open' if x < 10 else 'circle'
                    for x in game_nums[i]],
                name=slot.split('-')[1],
                hovertemplate=
                '<br><b>Position</b>: ' + slot.split('-')[1] +
                '<br><b>Tech</b>: %{y}' +
                '<br><b>Ave Score</b>: %{x}' +
                '<br><b>Num Found</b>: %{text}' +
                '<extra></extra>',
                text=game_nums[i]
            ))

        average_score_overall = 0
        for j in range(num_players):
            df = factions_pos[j]
            average_score_overall += df['pos_' + str(j + 1) + "_score"].mean()
        average_score_overall = average_score_overall / num_players

        fig.add_vline(x=average_score_overall, line_dash="dot", line_width=2,
                      annotation_text="overall average: " + str(round(average_score_overall, 2))
                      )

        fig.update_layout(
            title="Adv-Tech Impact on Score by Track: " + faction + " (score > 130, no dropped players, bid added)",
            xaxis_title="Average Score",
            yaxis_title="Advanced Tech")

        
        #fig.show()
        gitly.show(fig)

In [81]:
plot_adv_freq_remote(4, dat_4)

In [20]:
plot_adv_track_remote(4, dat_4)